# Import libraries

In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy

# Scrap URL

In [3]:
list_of_all_pages = []
for i in range(1,39):
    url = f'https://www.internsg.com/jobs/{i}/?f_p=107&f_i&filter_s#isg-top'
    list_of_all_pages.append(url)

In [9]:
data = []
for url in list_of_all_pages:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    divs = soup.find_all('div', class_='ast-col-lg-3')
    for div in divs:
    # Try to find an <a> tag within the <div>
        a_tag = div.find('a')
        # If an <a> tag is found and it has a 'href' attribute
        if a_tag and 'href' in a_tag.attrs:
            # Extract the URL and the job title
            url = a_tag['href']
            job_title = a_tag.get_text().strip()
            # Append the data to the list
            date = 'Not Available'
            # Try to find the next sibling 'div' which might contain the date
            date_div = div.find_next_sibling('div', class_='ast-col-lg-1')
            if date_div:
                date_span = date_div.find('span', class_='text-monospace')
                if date_span:
                    date = date_span.get_text().strip()
            data.append({'URL': url, 'Job Title': job_title, 'Date':date})

title_url_df = pd.DataFrame(data)
    

In [11]:
title_url_df['Date'].unique()

array(['22 Mar', '21 Mar', '20 Mar', '18 Mar', '16 Mar', '15 Mar',
       'Closed', '14 Mar', '13 Mar', '12 Mar', '11 Mar', '10 Mar',
       '08 Mar', '07 Mar', '05 Mar', '04 Mar', '01 Mar', '29 Feb',
       '28 Feb', '27 Feb', '26 Feb', '23 Feb', '22 Feb', '20 Feb',
       '19 Feb', '15 Feb', '14 Feb', '09 Feb', '08 Feb', '07 Feb',
       '06 Feb', '02 Feb', '01 Feb', '31 Jan', '30 Jan', '29 Jan',
       '26 Jan', '25 Jan', '24 Jan'], dtype=object)

In [5]:
title_url_df.to_csv('title_url_with_date.csv', index=False)

# Srap job details

In [6]:
col_names = ['Company','Designation','Date Listed','Job Type','Job Period','Profession',
             'Industry','Location Name','Allowance / Remuneration','Company Profile',
             'Job Description']
jobs_info = []
for url in title_url_df['URL']:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    dict = dict.fromkeys(col_names, '')
    for col_name in col_names:
        # Find the div that contains the column name
        col_div = soup.find('div', text=col_name, class_='font-weight-bold')
        if col_div:
            # The actual data is in the next sibling of the parent of col_div
            next_div = col_div.find_next_sibling()
            if next_div:
                # Extract the text and store it in the dictionary
                for span in next_div.find_all('span'):
                    span.decompose()
                dict[col_name] = next_div.get_text(strip=True)
    jobs_info.append(dict)
internSG_jobs = pd.DataFrame(jobs_info)

In [7]:
internSG_jobs.to_csv('internSG_jobs.csv',index=False)